In [2]:
import numpy as np
from numpy.fft import fft2, fft


In [26]:
N = 10
x = np.random.random(N).reshape((N//2, 2))
x

array([[0.26313009, 0.34539637],
       [0.15243536, 0.02044155],
       [0.59232609, 0.25631536],
       [0.07035365, 0.42501981],
       [0.9335804 , 0.99084113]])

### axis enumeration:
axis=0   is up/down, ei. change of rows

axis=1   is left/right, ei. change of cols

In [31]:
res2 = fft2(x)
print(res2)


[[ 4.0498398 +0.j         -0.02618864+0.j        ]
 [ 0.16929643+1.45817221j -0.04407993-0.58596146j]
 [-0.67290019+1.36550992j -0.14849145+0.54563173j]
 [-0.67290019-1.36550992j -0.14849145-0.54563173j]
 [ 0.16929643-1.45817221j -0.04407993+0.58596146j]]


In [28]:
def myfunc(x):
    return fft(x)

fft2 can be calculated by
first applying fft on each matrix row
then applying fft on that

In [32]:
# slower onEachRow = np.apply_along_axis(func1d=myfunc, axis=1, arr=x)
onEachRow = fft(x, axis=1)
#print(onEachRow)

# slower onEachCol = np.apply_along_axis(func1d=myfunc, axis=0, arr=onEachRow)
# np.apply_along_axis is slower because it uses python loops under the hood
onEachCol = fft(onEachRow, axis=0)
print(onEachCol)
print("good results? ",np.allclose(onEachCol, res2))

[[ 4.0498398 +0.j         -0.02618864+0.j        ]
 [ 0.16929643+1.45817221j -0.04407993-0.58596146j]
 [-0.67290019+1.36550992j -0.14849145+0.54563173j]
 [-0.67290019-1.36550992j -0.14849145-0.54563173j]
 [ 0.16929643-1.45817221j -0.04407993+0.58596146j]]
good results?  True


In [33]:
def myfft2(x):
    ''' Computes 2d fft using numpys fft '''
    onEachRow = fft(x, axis=1)
    return fft(onEachRow, axis=0)


In [39]:
t = np.arange(1000).reshape(100,10)
%timeit f2 = fft2(t)
print('Surprisingly, myfft2 is faster than fft2')
%timeit f1 = myfft2(t)
print(np.allclose(myfft2(t), fft2(t)))

42.4 µs ± 616 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Surprisingly, myfft2 is faster than fft2
30.8 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
True


In [109]:
def myfft_bad(x):
    x = x.astype(float)
    N = x.shape[0]
    n = np.arange(N).reshape((N,1))
    k = n.reshape((1,N))
    # k and n make an N x N matrix of 
    # complex coefficients that get multiplied with x
    Mat = np.exp(np.pi * -2j * k * n/ N)
    return np.dot(Mat, x)

In [120]:
t=np.arange(4000)

v1 = myfft_bad(t)
v2 = fft(t)
#print("they're insignificantly differenc",v1-v2)
np.allclose(v1,v2)

True

In [122]:
# at len(t)=3000 myff_bad takes 614ms, and at 4k it takes 1.1s
# while np.fft only takes 63µs 
%timeit myfft_bad(t)
%timeit fft(t)
print('myfft_bad is a lot slower than npfft')
print('myfft_bad has O(N^2) numerical operations while np.fft has O(NlogN)')

1.09 s ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
62.8 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
myfft_bad is a lot slower than npfft
myfft_bad has O(N^2) numerical operations while np.fft has O(NlogN)


In [123]:
#testing types
test = np.array([1,2,3])
test2 = test.astype(float)
print(test.dtype)
print(test2.dtype)

a = test.reshape(3,1)
print(a.shape)
a = test.reshape((3,1))
print(a.shape)

arr = np.arange(10)
print(arr.shape)
arr = np.arange(10).reshape(10,1)
print(arr.shape)

n = np.arange(10)
k = n.reshape((10, 1))
#print(n*k)
n = np.arange(10).reshape((10,1))
k = n.T
#print(n*k)
#print(n*n.T)

int32
float64
(3, 1)
(3, 1)
(10,)
(10, 1)


In [ ]:
print("TODO better implementation for fft")